Create and environment to use with this notebook:<br>
conda create -n ml3_final_project scipy scikit-image statsmodels scikit-learn pandas ipykernel tqdm keras pillow matplotlib pytorch==2.4.1 pytorch-cuda=12.4 torchinfo tensorflow=2.17 accelerate langchain pydub -c pytorch -c nvidia -c conda-forge

Install prerequisites uncomment and run once

In [ ]:
# ! pip install parler-tts
# ! pip install spaces
# ! pip install accelerate
# ! pip install pydub
# ! pip install transformers
# # ! pip install --upgrade protobuf
# ! pip install openai-whisper
# !pip install langchain_openai
# !pip install langchain_deepseek
# !pip install gtts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 844.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.7/100.7 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 

Setup The LLM Part

In [ ]:
# from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime
import random

In [ ]:
conversation_history = []
intro_given = False
active = False
inactive_timer = datetime.now()

In [ ]:
def get_response(query):
    global intro_given
    confused_messages = [
        "I'm sorry, I did not understand. Can you repeat that please?",
        "Paumanhin, hindi ko maintindihan ang iyong sinabi. Maari mo bang ulitin?",
        "Sorry, I did not understand. Can you repeat that last message please?",
        "I'm sorry, I did not quite get that. Can you repeat that please?",
        "Sorry, I did not quite get that. Can you repeat that please?",
        "I'm sorry, I did not quite get that. Can you repeat please?",
        "I'm sorry, I did not get that. Can you repeat that last message please?",
    ]


    if query == None:
        return None

    template = """Question: {question}

    Answer: """

    prompt = PromptTemplate.from_template(template)
    # llm =ChatDeepSeek(
    #         model="deepseek-chat",
    llm =ChatOpenAI(
            model="gpt-4o",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2,
            # other params...
        )
    llm_chain = prompt | llm
    if len(conversation_history) > 20:
        del conversation_history[0]

    intro_intsruction = ''

    if not intro_given:
        intro_intsruction ='Introduce yourself to the patient.'

    lang_detect = f"what language is the following text? give a one word answer: {query}"
    language = llm_chain.invoke(lang_detect).content
    print(f'LANGUAGE={language}')
    if (language.strip().lower() not in ['english', 'filipino', 'tagalog']):
        response = random.choice(confused_messages)
        interaction = {'patient':query, 'you':response}
        conversation_history.append(interaction)
        return response
    question = f"You are Chloe, Doctor Mike's assistant interviewing a patient. {intro_intsruction} Only ask one question at a time if needed.  Ask around 5 questions then tell the patient that you will relay the information to the doctor. You can give a summary of the conversation whan asked. Do not give answers as lists, and do not use any markup. Do not use apostrpohes to shorten words. Use word representations for numbers. Do not abbreviate words. Give answers in {language}. Give a summary of the conversaion to the patient after you have asked your questions. Conversaion history is:{conversation_history} The current query is: {query}"
    print(f'conversation history size={len(conversation_history)} question length={len(question)}')
    response = llm_chain.invoke(question).content
    interaction = {'patient':query, 'you':response}
    conversation_history.append(interaction)

    intro_given = True
    intro_intsruction = ''

    return response

Setup Whisper Speech To Text

In [ ]:
from transformers import pipeline
import gradio as gr
import torch
import whisper

if torch.cuda.is_available():
  device = "cuda:0"
else:
  device = "cpu"
# print("Using device", device)
model = whisper.load_model("base")
model.to(device)
# wake the model up

print(f'WHISPER SAYS {model.transcribe("./hello.wav")["text"]}')

# pipe = pipeline(model=model, device=device , return_timestamps=True)

# def transcribe(audio):
#     if audio == None:
#       #  print('NO DATA')
#        response = 'NO DATA GIVEN'
#        return response
#     try:

#       text = model.transcribe(audio, language="tl")["text"]
#       if len(text.strip()) > 0:
#         print(f'TRANSCRIBE {text}')
#       return text
#     except:
#        response = 'ERROR TRANSCRIBING'
#        return response

/home/andy/miniconda3/envs/ml3_final_project/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-20 13:53:10.039852: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-20 13:53:10.067164: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-20 13:53:10.077376: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-20 13:53:10.102424: I tensorflow/core/platform/cpu_feature_guard.cc:210] This 

WHISPER SAYS  Hello?


In [ ]:


import soundfile as sf
import librosa
import numpy as np


talking = False
silence_counter = 0
voice_data=np.array([])
last_query = ''
last_response=''

def transcribe(data, orig_rate):
    audio_array = librosa.resample(data, orig_sr=orig_rate, target_sr=16000)
    norm = np.linalg.norm(audio_array)
    if norm != 0:
        audio_array = audio_array / norm
    # sf.write('new_file.wav', audio_array, orig_rate)
    audio_array =  torch.from_numpy(audio_array.astype(np.float32))
    audio_array.to(device)
    # print(np.abs(data).mean())
    result = model.transcribe(audio_array,language="tl")
    print(result)
    return result['text']

def detect_signal(data):
    global talking
    global silence_counter
    global voice_data
    global last_query
    global last_response
    global active
    global inactive_timer
    result = None

    silence_amplitude_threshold = 500
    silence_time_treshold = 2
    mean = np.mean(np.abs(data[1]))
    # print(mean)
    if mean > silence_amplitude_threshold:
        talking = True
        silence_counter=0
        voice_data = np.concatenate([voice_data,data[1]])
        # print(f'SAMPLE RATE={data[0]} VOICE DATA SHAPE={voice_data.shape}')

    elif talking:
        silence_counter += 1

    if silence_counter > silence_time_treshold:
        silence_counter = 0
        talking = False
        result = voice_data
        voice_data = np.array([])

    # print(f'SILENCE {silence_counter}')

    if type(None) != type(result):
        transcription = transcribe(result, data[0])
        if not active and transcription.lower().replace(',','').strip().startswith('hey chloe'):
            last_query = transcription
            active = True
            inactive_timer = datetime.now()
        elif active == True:
            inactive_timer = datetime.now()
            last_query = transcription
    else:
        inactive_time =  datetime.now() - inactive_timer
        print(f'INACTIVE TIME {inactive_time.total_seconds()}')
        if inactive_time.total_seconds() > 60:
            active = False
    return last_query

Setup the Parler Text To Speech Model

In [ ]:
# from parler_tts import ParlerTTSStreamer
# from parler_tts import ParlerTTSForConditionalGeneration

# from transformers import AutoTokenizer, AutoFeatureExtractor, set_seed
# import numpy as np
# import spaces
# import torch
# from threading import Thread


# device = "cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
# torch_dtype = torch.float16 if device != "cpu" else torch.float32

# modelid_tiny = "parler-tts/parler-tts-tiny-v1"
# print('LOADING PARLER TTS MODEL')
# tts_model = ParlerTTSForConditionalGeneration.from_pretrained(
#     modelid_tiny, torch_dtype=torch_dtype, low_cpu_mem_usage=True
# ).to(device)

# tokenizer = AutoTokenizer.from_pretrained(modelid_tiny)
# feature_extractor = AutoFeatureExtractor.from_pretrained(modelid_tiny)

# sampling_rate = tts_model.audio_encoder.config.sampling_rate
# frame_rate = tts_model.audio_encoder.config.frame_rate
# description = "Jenna speaks at an average pace with a calm delivery in a very confined sounding environment with clear audio quality."
# description_tokens = tokenizer(description, return_tensors="pt").to(device)

# print('READY')
# SAMPLE_RATE = feature_extractor.sampling_rate
# SEED = 42


Flash attention 2 is not installed


LOADING PARLER TTS MODEL


/home/andy/miniconda3/envs/ml3_final_project/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "google/flan-t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "output_past": true,
  "pad_token_id": 0,
  "r

READY


In [ ]:

from pydub import AudioSegment
import io
import re

def numpy_to_mp3(audio_array, sampling_rate):
    # Normalize audio_array if it's floating-point
    if np.issubdtype(audio_array.dtype, np.floating):
        max_val = np.max(np.abs(audio_array))
        audio_array = (audio_array / max_val) * 32767 # Normalize to 16-bit range
        audio_array = audio_array.astype(np.int16)

    # Create an audio segment from the numpy array
    audio_segment = AudioSegment(
        audio_array.tobytes(),
        frame_rate=sampling_rate,
        sample_width=audio_array.dtype.itemsize,
        channels=1
    )

    # Export the audio segment to MP3 bytes - use a high bitrate to maximise quality
    mp3_io = io.BytesIO()
    audio_segment.export(mp3_io, format="mp3", bitrate="320k")

    # Get the MP3 bytes
    mp3_bytes = mp3_io.getvalue()
    mp3_io.close()

    return mp3_bytes

# sampling_rate = tts_model.audio_encoder.config.sampling_rate
# frame_rate = tts_model.audio_encoder.config.frame_rate

def split_text(text):
    text = text.replace('(','').replace(')','')
    phrases = re.split(r'(\.\s+|\n|\?\s+|\!\s+|\:\s+)', text)
    # print(phrases)
    reconstructed = [ ''.join(x) for x in zip(phrases[0::2], phrases[1::2])]
    reconstructed.append(phrases[-1])
    stripped = [x.strip().replace('.','').replace('!','') for x in reconstructed if len(x.strip()) > 1]
    # print(reconstructed)
    return stripped

In [ ]:
# @spaces.GPU
# def read_response(answer):
#     if answer == None:
#         return None

#     print(f'READING RESPONSE {answer}')

#     play_steps_in_s = 5.0
#     play_steps = int(frame_rate * play_steps_in_s)


#     phrases = split_text(answer)

#     for phrase in phrases:
#         streamer = ParlerTTSStreamer(tts_model, device=device, play_steps=play_steps)
#         prompt = tokenizer(phrase, return_tensors="pt").to(device)

#         generation_kwargs = dict(
#             input_ids=description_tokens.input_ids,
#             prompt_input_ids=prompt.input_ids,
#             streamer=streamer,
#             do_sample=True,
#             temperature=1.0,
#             min_new_tokens=20,
#         )

#         set_seed(42)
#         thread = Thread(target=tts_model.generate, kwargs=generation_kwargs)
#         thread.start()

#         for new_audio in streamer:
#             print(f"Sample of length: {round(new_audio.shape[0] / sampling_rate, 2)} seconds")
#             yield phrase, numpy_to_mp3(new_audio, sampling_rate=sampling_rate)

In [ ]:
from gtts import gTTS
from io import BytesIO
from pydub import AudioSegment

def read_response_gtts(answer):
    if answer == None or answer=='':
        return None
    print(f'READING RESPONSE {answer}')
    language = 'tl'

    phrases = split_text(answer)

    for phrase in phrases:
        print(phrase)
        myMP3 = BytesIO()
        tts = gTTS(text=phrase, lang=language, slow=False)
        tts.write_to_fp(myMP3)
        myMP3.seek(0)
        yield myMP3.getvalue()

In [ ]:
def update_output():
    print('UPDATE')

Launch the Web Interface

In [ ]:
with gr.Blocks() as block:
    gr.HTML(
        f"""
        <h1 style='text-align: center;'>Cutiepies Healthcare Assistant Chatbot</h1>
        <h3 style='text-align: center;'>Start with "Hey, Chloe!" Then ask a question.</h3>
        """
    )
    with gr.Group():
        with gr.Row():
            audio_out = gr.Audio(label="Spoken Answer", streaming=True, autoplay=True)
            query = gr.Textbox(label="Query")
            answer = gr.Textbox(label="Answer")
            # state = gr.State()
        with gr.Row():
            audio_in = gr.Audio(label="Speak your question", sources="microphone", streaming=True, type="numpy")

    audio_in.stream(detect_signal, inputs = audio_in, outputs = [query])
    query.change(get_response,inputs=query, outputs=[answer])
    answer.change(read_response_gtts,inputs=answer, outputs=[audio_out])



block.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


INACTIVE TIME 27.817979
INACTIVE TIME 28.273051
INACTIVE TIME 28.807498
INACTIVE TIME 29.316372
INACTIVE TIME 29.831965
INACTIVE TIME 30.265165
{'text': ' Hey Chloe.', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 1.0, 'text': ' Hey Chloe.', 'tokens': [50364, 1911, 29694, 13, 50414], 'temperature': 0.0, 'avg_logprob': -0.81099534034729, 'compression_ratio': 0.5555555555555556, 'no_speech_prob': 0.1689653843641281}], 'language': 'tl'}
INACTIVE TIME 0.003219
INACTIVE TIME 0.506786
INACTIVE TIME 0.907907
LANGUAGE=English
conversation history size=0 question length=624
INACTIVE TIME 1.424092
INACTIVE TIME 1.941154
READING RESPONSE Hello, I am Chloe, Doctor Mikes assistant. How are you feeling today?
Hello, I am Chloe, Doctor Mikes assistant
INACTIVE TIME 2.467079
How are you feeling today?
INACTIVE TIME 2.975714
INACTIVE TIME 3.507926
INACTIVE TIME 3.924399
INACTIVE TIME 4.442868
INACTIVE TIME 4.957013
INACTIVE TIME 5.473702
INACTIVE TIME 5.989879
INACTIVE TIME 6.410266
INACTIVE T